In [1]:
import pandas as pd 
import numpy as np
from sentence_transformers import SentenceTransformer
from llama_index.embeddings.adapter_utils import BaseAdapter
from llama_index.embeddings.adapter_utils import TwoLayerNN
from llama_index.embeddings import AdapterEmbeddingModel
from llama_index.embeddings import resolve_embed_model
import torch
import torch.nn.functional as F
from torch import nn, Tensor
from typing import Dict
from configparser import ConfigParser
from openai import OpenAI
from src.embeddings_utils import *
pd.set_option('display.max_colwidth',-1)

/tmp/ipykernel_122/1607660176.py:15: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth',-1)


## Load data

In [2]:
df=pd.read_excel('data/riigikogu_w_meta/data_all.xlsx').tail(10000)
df.shape

(10000, 23)

In [3]:
df=df.reset_index()

In [4]:
texts=df.text_splitted

In [5]:
embs_ar=np.load('data/riigikogu_w_meta/embs_all.npy', allow_pickle=True)[-10000:]
embs_ar.shape

(10000,)

# load models

In [6]:
model_orig=SentenceTransformer("intfloat/multilingual-e5-base")
model_fintuned = SentenceTransformer('test_model')

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/200 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/179k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/280 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/387 [00:00<?, ?B/s]

In [7]:
base_embed_model = resolve_embed_model("local:intfloat/multilingual-e5-base")

# #2-layer model
# adapter_model = TwoLayerNN(
#     768,  # input dimension
#     1024,  # hidden dimension
#     768,  # output dimension
#     bias=True,
#     add_residual=True,
# )

# embed_model_2layer = AdapterEmbeddingModel(
#     base_embed_model,
#     "modele5_output_test",
#     adapter_model,
# )


Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [8]:
#custom adapter model
class CustomNN(BaseAdapter):
    """Custom NN transformation.

    Is a copy of our TwoLayerNN, showing it here for notebook purposes.

    Args:
        in_features (int): Input dimension.
        hidden_features (int): Hidden dimension.
        out_features (int): Output dimension.
        bias (bool): Whether to use bias. Defaults to False.
        activation_fn_str (str): Name of activation function. Defaults to "relu".

    """

    def __init__(
        self,
        in_features: int,
        hidden_features: int,
        out_features: int,
        bias: bool = False,
        add_residual: bool = False,
    ) -> None:
        super(CustomNN, self).__init__()
        self.in_features = in_features
        self.hidden_features = hidden_features
        self.out_features = out_features
        self.bias = bias

        self.linear1 = nn.Linear(in_features, hidden_features, bias=True)
        self.linear2 = nn.Linear(hidden_features, out_features, bias=True)
        self._add_residual = add_residual
        # if add_residual, then add residual_weight (init to 0)
        self.residual_weight = nn.Parameter(torch.zeros(1))

    def forward(self, embed: Tensor) -> Tensor:
        """Forward pass (Wv).

        Args:
            embed (Tensor): Input tensor.

        """
        output1 = self.linear1(embed)
        output1 = F.relu(output1)
        output2 = self.linear2(output1)

        if self._add_residual:
            output2 = self.residual_weight * output2 + embed

        return output2

    def get_config_dict(self) -> Dict:
        """Get config dict."""
        return {
            "in_features": self.in_features,
            "hidden_features": self.hidden_features,
            "out_features": self.out_features,
            "bias": self.bias,
            "add_residual": self._add_residual,
        }

    
custom_adapter = CustomNN(
    768,  # input dimension
    2048,  # hidden dimension
    768,  # output dimension
    bias=True,
    add_residual=True,
)

In [9]:
embed_model_custom = AdapterEmbeddingModel(
    base_embed_model,
    "custom_modele5_output_test",
    custom_adapter,
)
    

## Embed texts with different models

In [10]:
%time embs_orig = model_orig.encode(texts.tolist())

CPU times: user 14.6 s, sys: 4.48 s, total: 19.1 s
Wall time: 17.6 s


In [11]:
%time embs_finet_2l = model_fintuned.encode(texts.tolist())

CPU times: user 14.1 s, sys: 4.43 s, total: 18.6 s
Wall time: 15.5 s


In [12]:
%time embs_train_finet_cust=[embed_model_custom.get_text_embedding(t) for t in texts]

CPU times: user 1min 18s, sys: 187 ms, total: 1min 18s
Wall time: 1min 18s


## open ai model

In [13]:
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

#setup openai client
config=ConfigParser()
config.read('conf/conf.ini')
client = OpenAI(api_key=config['openai']['apikey'])

def get_embeddings_openai(texts, model="text-embedding-ada-002"):
    texts = [text.replace("\n", " ") for text in texts]
    response= client.embeddings.create(input = texts, model=model)
    idxs=[el.index for el in response.data]
    embs=[el.embedding for el in response.data]
    
    results=[(texts[i_orig], embs[i_resp]) for  i_resp, i_orig in enumerate(idxs)]
    return results

# search texts

In [14]:
len(list(enumerate(np.array(embed_model_custom.get_text_embedding(['tere'])).reshape((1, -1)))))

1

In [15]:
def get_embeddings(texts, embed_model):
    texts = [text.replace("\n", " ") for text in texts]
    try:
        embeddings = embed_model.encode(texts)
    except:
        embeddings= np.array(embed_model.get_text_embedding(texts)).reshape((1, -1))
    # print(embeddings)
    # print(len(embeddings))
    results=[(texts[i], emb) for  i, emb in enumerate(embeddings)]
    return results


def sim_score(review_embedding, search_embeddings):
    # print(search_embeddings[0])
    return cosine_similarity(review_embedding, search_embeddings[0][1])


def get_search_sim_score(df, embs, search_text, embed_model=False, openai=False):
    if openai:
        search_emb=get_embeddings_openai([search_text])
    else:
        search_emb=get_embeddings([search_text], embed_model)
    scores=[]
    for i, row in df.iterrows():
        if i%10000==0:
            print(i)
        emb=embs[i]
        score=sim_score(emb, search_emb)
        scores.append(score)
    df['search_score']=scores
    return df

In [16]:
search_text='keskmaa õhutõrje'
df_finet_2l=get_search_sim_score(df, embs_finet_2l, search_text, model_fintuned)

0


In [17]:
df_finet_2l=df_finet_2l.sort_values('search_score', ascending=False)
df_finet_2l[['text_splitted', 'search_score']].head()

,text_splitted,search_score
8374,keskmaa-õhutõrjesüsteemi iris-t ostmiseks saksa kaitsetööstusettevõttelt diehl defence see maksab umbes 400 miljonit eurot tegu on suurima eesti ajaloo kaitsevaldkonna lepinguga ja see suurendab oluliselt strateegiliste maa-alade õhukaitset lisaks hangime mitmikraketiheitjad himars mis läheb maksma 200 miljonit eurot ja täiendavad liikursuurtükid k9 12 suurtükki koos täiendava varustuse laskemoonaga circa 100 miljonit uutest võimearendustest on käimas teise jalaväebrigaadi soomustamine ehk manööverpataljonid saavad veoautode asemel soomukid brigaadi tankitõrjekompaniid saavad täiendavad laskeseadmed ning soomusautod millega tagatakse nende parem kaitstus ja mobiilsus mereväes uuendatakse kõik,0.455836
5881,avan läbirääkimised eesti keskerakonna fraktsiooni nime andre hanimägi palun,0.426103
2614,aitäh anastassia kovalenko-kõlvart kas keskfraktsiooni nimel palun,0.423893
5733,aitäh järgnevalt helir-valdor seeder palun,0.397794
1565,aitäh järgnevalt helir-valdor seeder palun,0.397794


In [77]:
search_text='keskmaa õhutõrje'
df_finet_cust=get_search_sim_score(df, embs_train_finet_cust, search_text, embed_model_custom)

0


In [78]:
df_finet_cust=df_finet_cust.sort_values('search_score', ascending=False)
df_finet_cust[['text_splitted', 'search_score']].head()

,text_splitted,search_score
399,kaitsta ja hoida pikaaegseid piirivalveohvitsere selle asemel et anda ülesanne konkreetsele ministeeriumile lahendada,0.886749
8047,aitäh minu arvates kas saaks ilma lärmita,0.886004
3017,oluliselt kasvanud kulud sealhulgas märkimisväärselt kaitsekulud,0.885112
2730,tegevuskulude kärpimise ülesanded et lihtsam oleks seda läbi viia milline kärbe tuleb tegelikult riigihalduses konkreetselt,0.883719
9690,et nad ei pea lihtsalt vajalikuks kohale tulla ja ei ilmunud,0.883192


In [16]:
search_text='keskmaa õhutõrje'
df_orig=get_search_sim_score(df, embs_orig, search_text, model_orig)

0


In [17]:
df_orig=df_orig.sort_values('search_score', ascending=False)
df_orig[['text_splitted', 'search_score']].head()

,text_splitted,search_score
5881,avan läbirääkimised eesti keskerakonna fraktsiooni nime andre hanimägi palun,0.843727
9964,haigla kus neile maailmatasemel tervishoiuteenust osutatakse,0.841727
4858,kolm minutit kolm minutit lisaaega,0.840080
8374,keskmaa-õhutõrjesüsteemi iris-t ostmiseks saksa kaitsetööstusettevõttelt diehl defence see maksab umbes 400 miljonit eurot tegu on suurima eesti ajaloo kaitsevaldkonna lepinguga ja see suurendab oluliselt strateegiliste maa-alade õhukaitset lisaks hangime mitmikraketiheitjad himars mis läheb maksma 200 miljonit eurot ja täiendavad liikursuurtükid k9 12 suurtükki koos täiendava varustuse laskemoonaga circa 100 miljonit uutest võimearendustest on käimas teise jalaväebrigaadi soomustamine ehk manööverpataljonid saavad veoautode asemel soomukid brigaadi tankitõrjekompaniid saavad täiendavad laskeseadmed ning soomusautod millega tagatakse nende parem kaitstus ja mobiilsus mereväes uuendatakse kõik,0.839920
2590,helistab kella. head kolleegid palun vaikust,0.839131


In [83]:
df[df.text_splitted.str.contains('õhutõrje')].shape

(3, 25)

In [85]:
search_text='keskmaa õhutõrje'
df_oai=get_search_sim_score(df, embs_ar, search_text, openai=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
0


In [86]:
df_oai=df_oai.sort_values('search_score', ascending=False)
df_oai[['text_splitted', 'search_score']].head()

,text_splitted,search_score
9148,see on vaenu õhutamine see sõnavõtt need tegevused see on sõnavõtt ja see on vaenu õhutamine,0.872676
8374,keskmaa-õhutõrjesüsteemi iris-t ostmiseks saksa kaitsetööstusettevõttelt diehl defence see maksab umbes 400 miljonit eurot tegu on suurima eesti ajaloo kaitsevaldkonna lepinguga ja see suurendab oluliselt strateegiliste maa-alade õhukaitset lisaks hangime mitmikraketiheitjad himars mis läheb maksma 200 miljonit eurot ja täiendavad liikursuurtükid k9 12 suurtükki koos täiendava varustuse laskemoonaga circa 100 miljonit uutest võimearendustest on käimas teise jalaväebrigaadi soomustamine ehk manööverpataljonid saavad veoautode asemel soomukid brigaadi tankitõrjekompaniid saavad täiendavad laskeseadmed ning soomusautod millega tagatakse nende parem kaitstus ja mobiilsus mereväes uuendatakse kõik,0.867954
8351,aitäh lugupeetud istungi juhataja hea peaminister eesti on ka üks osa euroopast ja nagu me teame meie viimased talved on olnud väga külmad ja pikad kas seda nimetada kliima soojenemiseks või milleks on igaühe enda pädevuses aga teie jutust jäi kõlama et kohalikud omavalitsused ju peaksid toetama neid asju ja järjekordselt inimesi suunatakse toimetulekut taotlema on ta selle renoveerimistoetuse laenutaotluste ja intressimaksete ja kindlustusmaksete kompenseerimiseks ja samas on seal ette nähtud piirmäär toimetulekutoetusel nüüd me teame seda et ahjude vahetuse,0.854290
8333,mis veel ei kuulu kauplemissüsteemi ja mis kasutavad maagaasi selliseid katlamajasid on võrdlemisi vähe alles kuna enamik neist on hiljuti moderniseeritud ja kasutavad kütusena biomassi samas kuna soojusenergia tarbimine on vältimatu ka madalama sissetulekuga elanikele võib teatud osa elanike reaalne sissetulek langeda ulatuslikum mõju avaldub transpordikütuste kaudu mis on eestis veel väga suures osas fossiilset päritolu samas ligikaudu 50% eesti transpordisektori kasvuhoonegaaside heitest tekitab tallinn koos harjumaaga kus alternatiivide pakkumine isiklikule sõiduautole on võimalik ning inimeste ostujõud on kõrgem kui,0.851373
8927,"eesti kaitsevõime suureneb oluliselt nii maismaa- õhu- kui ka meredomeenis riigikaitse algab inimestest investeerime riigi kaitsjatesse ja nende varustusse jätkame 20 000 võitlejani suurendatud maakaitse võitlusvõime hoidmist testime meie ajateenistusel põhineva reservarmee ja maakaits valmidust kaitseliitu rahastame 2024. 2027. aastal ligi 300 miljoni euroga millest 227,9 miljonit on iga-aastane toetus ja lisaks maakaitset toetavad hanked 64,9 miljoni euro eest tänavu on esimene kord kui laiapindse julgeoleku rahastamiseks pani valitsus kokku eraldi kava mis hõlmas seitset ministeeriumit ja riigikantseleid uus formaat",0.850838


## Use some reranker

In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("amberoad/bert-multilingual-passage-reranking-msmarco")
model = AutoModelForSequenceClassification.from_pretrained("amberoad/bert-multilingual-passage-reranking-msmarco")

Downloading:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/696 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

In [19]:
from transformers import pipeline

pipe = pipeline("text-classification", model="amberoad/bert-multilingual-passage-reranking-msmarco")

In [23]:
pipe('tere')

[{'label': 'LABEL_0', 'score': 0.9768011569976807}]

In [31]:
texts=[]
scores=[]
for t in df_orig['text_splitted'].head().tolist():
    res=pipe(search_text+'. '+ t)
    print(t)
    print(res)

avan läbirääkimised eesti keskerakonna fraktsiooni nime andre hanimägi palun
[{'label': 'LABEL_0', 'score': 0.983137845993042}]
haigla kus neile maailmatasemel tervishoiuteenust osutatakse
[{'label': 'LABEL_0', 'score': 0.988508403301239}]
kolm minutit kolm minutit lisaaega
[{'label': 'LABEL_0', 'score': 0.9908918738365173}]
keskmaa-õhutõrjesüsteemi iris-t ostmiseks saksa kaitsetööstusettevõttelt diehl defence see maksab umbes 400 miljonit eurot tegu on suurima eesti ajaloo kaitsevaldkonna lepinguga ja see suurendab oluliselt strateegiliste maa-alade õhukaitset lisaks hangime mitmikraketiheitjad himars mis läheb maksma 200 miljonit eurot ja täiendavad liikursuurtükid k9 12 suurtükki koos täiendava varustuse laskemoonaga circa 100 miljonit uutest võimearendustest on käimas teise jalaväebrigaadi soomustamine ehk manööverpataljonid saavad veoautode asemel soomukid brigaadi tankitõrjekompaniid saavad täiendavad laskeseadmed ning soomusautod millega tagatakse nende parem kaitstus ja mobiils